In [35]:
import importlib
from sklearn.utils.estimator_checks import check_estimator
import vector_similarity
importlib.reload(vector_similarity)
from vector_similarity import VectorSimilarity
from sklearn.svm import SVC

In [36]:
check_estimator(VectorSimilarity())